<img align="right" src="images/DSApps_logo_small.jpg">

# DSApps 2023 @ TAU: Assignment 3

### Giora Simchoni

### Some Pandas and Web Scraping

### Welcome

Welcome to Assignment 3 in Python!

Remember:

* You can play with the assignment in Playground mode, but:
* Only your private Github repository assigned to you by the course admin will be cloned and graded (Submission mode, see instructions [here](https://github.com/DSApps-2023/Class_Slides/blob/main/Apps_of_DS_HW.pdf))
* Like any other University assignment, your work should remain private
* You need to `git clone` your private Github repository locally as explained [here](https://github.com/DSApps-2023/Class_Slides/blob/main/Apps_of_DS_HW.pdf)
* You need to uncomment the starter code inside the chunk, replace the `### YOUR CODE HERE ###`, run the chunk and see that you're getting the expected result
* Pay attention to what you're asked to do and the required output
* For example, using a *different* function than the one you were specifically asked to use, will decrease your score (unless you amaze me)
* Your notebook should run smoothly from start to end if someone presses in the Jupyter toolbar Kernel --> Restart & Run All
* When you're done save the entire notebook into a html file, this is the file that would be graded
* You can add other files but do not delete any files
* Commit your work and push to your private Github repository as explained [here](https://github.com/DSApps-2023/Class_Slides/blob/main/Apps_of_DS_HW.pdf)

This assignemtnt is due: 8/5 23:59

### Libraries

These are the libraries you will need. If you don't have them, you need to uncomment the `!pip install` line and install them first (you can also just copy this command to a terminal and do it there if you don't want all the output printed in this notebook).

In [2]:
#!pip install matplotlib numpy scipy pandas beautifulsoup4

In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Some Pandas

###### (20 points)

Behold the `fifa_wide` DF. There's only so much we can do with it in current form. Make `fifa_wide` into `fifa_long`, a table with 4 columns (say `['Country', 'Continent', 'Year', 'Rank']`) and 15 rows, holding for each country, for each World Cup year, its Rank.

In [88]:
fifa_d = {'Country': ['France', 'Germany', 'Spain', 'Argentina', 'Brazil'],
      'Continent': ['Europe', 'Europe', 'Europe', 'South America', 'South America'],
      '2014': [7, 1, 23, 2, 4],
      '2018': [1, 22, 10, 16, 6],
      '2022': [2, 17, 13, 1, 7]}

fifa_wide = pd.DataFrame(fifa_d)
fifa_wide

,Country,Continent,2014,2018,2022
0,France,Europe,7,1,2
1,Germany,Europe,1,22,17
2,Spain,Europe,23,10,13
3,Argentina,South America,2,16,1
4,Brazil,South America,4,6,7


In [89]:
fifa_long = pd.melt(fifa_wide,id_vars=('Country','Continent'),
                    value_vars=['2014','2018','2022'], 
                    var_name="Year", 
                    value_name="Rank")
fifa_long

,Country,Continent,Year,Rank
0,France,Europe,2014,7
1,Germany,Europe,2014,1
2,Spain,Europe,2014,23
3,Argentina,South America,2014,2
4,Brazil,South America,2014,4
5,France,Europe,2018,1
6,Germany,Europe,2018,22
7,Spain,Europe,2018,10
8,Argentina,South America,2018,16
9,Brazil,South America,2018,6


We have downloaded from [Gapminder data repository](https://www.gapminder.org/data/) the life expectancy data for all countries in the years 1900-1999. 

* Upload the `life_expectancy_years.csv` file from Moodle, however you want.
* Extract from this dataset countries that have data for all years in the range 1900-1999 (i.e. you should discard countires with `Nan` values in years 1900-1999).
* You should get 184 countries.
* Create for each of these the average age for each decade (1900-1909, 1910-1919,...), resulting with a matrix of 184 countries (rows) times 10 decades averages (the country's name is an extra column, so 11 columns).


**Important:** There are many ways to do this manipulation. Looping isn't a sin, but you should know where to use it.

**Sanity check:** the maximum value at the 90s (1990-1999) should be about `80.11`

In [90]:
df_pop = pd.read_csv('data/life_expectancy_years.csv')

In [91]:
df_pop = df_pop.filter(regex='^19|country').dropna()
d = {f"{i}0s_mean" : df_pop.loc[:,str(1900+10*i):str(1909+10*i)].mean(axis=1) for i in range(10)}
df_pop2 = pd.DataFrame(d)
df_pop2.insert(0,'country',df_pop['country'])

In [93]:
#Check for maximum value at the 90s - should be 80.11
df_pop2['90s_mean'].max()

80.10999999999999

**Note**: the following 3 questions are regarding the dataset you just created, not the original raw data from CSV.

Which country had the highest life expectancy in the 1970s (1970-1979) ?

In [94]:
df_pop2.loc[df_pop2['70s_mean'].idxmax()]['country']

'Greece'

How many countries had at least 1 "peak" decade? A peak decade is a decade which had a higher life expectancy compared to both the deacdes before and after it (the 1900s and 1990s cannot be called that here)

In [95]:
def peak(row):
    for i in range(2,10):
        if row[i-1] < row[i] > row[i+1]:
            return True
            break
df_pop2['peak'] = df_pop2.apply(peak,axis=1)
df_pop2['peak'].sum()

72

In which decade did most countries experience a decrease in life expectancy compared to the previous decade? (see if you can think of a reason) E.g. if you check from 1980s to 1990s about 20% of countries decreased in life expectency.

In [113]:
dec = np.empty(9)
for i in range(1,10):
    x = df_pop2.apply(lambda row: row[i] > row[i+1],axis=1)
    dec[i-1] = x.sum()
argmax = dec.argmax()
print(f"The decade that most countries exprience a decrease in life expectancy is {argmax+1}0s.\nProbably due to the spanish flu")

The decade that most countries exprience a decrease in life expectancy is 10s.
Probably due to the spanish flu


### Web Scraping with BeautifulSoup

###### (30 points)

Go to [ebay.com](ebay.com) and search for little boys t-shirts. The ebay website, like any modern website, is filled with text, images and links. But if you are using Google Chrome and you right-click on any page and choose "View page source" you will see the raw HTML script behind it. The following code was used to download thousands of boys and girls shirts images from ebay, your mission is to fill in the blanks.

Run the following piece of code

In [11]:
from bs4 import BeautifulSoup
import requests

url = 'https://il.ebay.com/b/Boys-Tops-Shirts-T-Shirts-for-Boys/260966/bn_1642883?rt=nc&LH_ItemCondition=1000&LH_BIN=1&LH_PrefLoc=3&_pgn=1'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

The above code imports Beautiful Soup, imports the requests library for handling web connections, assigns an ebay search results page address to a variable called `url`, "requests" this URL, stores the response in a variable called `r`, makes a `BeautifulSoup` object out of the response's `content`, and assigns it to a variable called `soup`.

It is advised to visit the url using your browser, so you will have a visual understanding of what you are doing.

Print the raw HTML:

In [12]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<!--[if IE 9]><html class="ie9" lang="en"><![endif]-->
<!--[if gt IE 9]><!-->
<html lang="en">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="Shop eBay for great deals on Boys Tops, Shirts &amp; T-Shirts for Boys. You'll find new or used products in Boys Tops, Shirts &amp; T-Shirts for Boys on eBay. Free shipping on selected items." name="description">
   <title>
    Boys Tops, Shirts &amp; T-Shirts for Boys | eBay
   </title>
   <meta content="unsafe-url" name="referrer"/>
   <meta content="Boys Tops, Shirts &amp; T-Shirts for Boys | eBay" property="og:title"/>
   <link href="https://ir.ebaystatic.com" rel="'preconnect'"/>
   <meta content="34E98E6F27109BE1A9DCF19658EEEE33" name="msvalidate.01"/>
   <meta content="Shop eBay for great deals on Boys Tops, Shirts &amp; T-Shirts for Boys. You'll find new or used products in Boys Tops, Shirts &amp; T-Shirts for Boys on eBay. Free shipping on selected items." property="og:description"/>
   <meta content

What is the page's title? Replace `#### your code here ####` to get the title as a simple string, without html tags.

In [14]:
url_title = soup.title.text
print(url_title)

Boys Tops, Shirts & T-Shirts for Boys | eBay


Find the all hyperlinks on page:

In [15]:
links_objs = soup.find_all('a', href=True)
all_links = [x['href'] for x in links_objs[1:]]
print('all_links is a: ' + str(type(all_links)))
print()
print('first 5 elements in all_links:')
print(all_links[:5])

all_links is a: <class 'list'>

first 5 elements in all_links:
['https://www.ebay.com/', 'http://www.ebay.com/sch/ebayadvsearch', 'https://signin.ebay.com/ws/eBayISAPI.dll?SignIn&_trksid=m570.l3348', 'https://www.ebay.com/globaldeals', 'https://ocsnext.ebay.com/ocs/home']


What is the class type of `all_links`?

In [16]:
print('all_links is a: ' + str(type(all_links)))

all_links is a: <class 'list'>


Find the all images on page (no credit):

In [15]:
images = soup.find_all("img")

[<img alt="eBay Logo" height="200" id="gh-logo" src="https://ir.ebaystatic.com/rs/v/fxxj3ttftm5ltcqnto1o4baovyl.png" style="clip:rect(47px, 118px, 95px, 0px); position:absolute; top:-47px;left:0" width="250"/>,
 <img alt="Tommy Hilfiger Baby / Boys / Youth Cotton Polo Shirt NEW WITH TAG ALL Sizes !" class="s-item__image-img" src="https://i.ebayimg.com/thumbs/images/g/xVAAAOSwba1h-sDI/s-l300.jpg"/>,
 <img alt="Chivas Del Guadalajara Youth Centered Full Zipper S Grade Track Soccer Jacket" class="s-item__image-img" src="https://i.ebayimg.com/thumbs/images/g/fCMAAOSwpxJkEsIe/s-l300.jpg"/>,
 <img alt="NWT 100% Authentic VILEBREQUIN 100 % Cotton POLO - BLUE - BOYS - 4 Years" class="s-item__image-img" src="https://i.ebayimg.com/thumbs/images/g/xgYAAOSwRath7eHS/s-l300.jpg"/>,
 <img alt="Disney Boys' Toy Story Snug Fit Cotton Pajamas Size 4, 6, 8, 10" class="s-item__image-img" src="https://i.ebayimg.com/thumbs/images/g/1aYAAOSw~ZNjwlYj/s-l300.jpg"/>,
 <img alt="Boys toddler CHAMPION 2 piece out

Get a `list` of all image titles from the `images` object, **except for the first one**. Print that list.

Hint: `alt`

In [18]:
image_titles = [x['alt'] for x in images[1:]]
print(image_titles)

['Tommy Hilfiger Baby / Boys / Youth Cotton Polo Shirt NEW WITH TAG ALL Sizes !', 'Chivas Del Guadalajara Youth Centered Full Zipper S Grade Track Soccer Jacket', 'NWT 100% Authentic VILEBREQUIN 100 % Cotton POLO - BLUE - BOYS - 4 Years', "Disney Boys' Toy Story Snug Fit Cotton Pajamas Size 4, 6, 8, 10", 'Boys toddler CHAMPION 2 piece outfit sets shirts shorts all sizes colors', "Boy's School Uniform Short Sleeve Polo Shirt TAGLESS (Sizes, 4-20) NWT", 'Boys Dress Shirt Solid Long Sleeve Formal Kids Wedding Party Boy Size 5 -18 New', 'Roblox Boys Short Sleeve T-Shirt Officially Licensed Black Large 14/16', "Polo Ralph Lauren Kids' Golf Shirts White Stripe S 5", 'Boys toddler New balance 2 and 3 piece outfit sets shirts tank top shorts', 'Pajama R Us Boys 5 Pack Ribbed Tank Tops Size 2T/3T 4T/5T 4/5 6/8 10/12 14/16 18', "NEW BOYS LEVI'S GRAPHIC TEE T SHIRT 2 PIECE PACK SET", 'BNWT Boys YLG Blue UNDER ARMOUR FREEDOM T Shirt (B1)', 'Nike Dri-Fit Short Sleeve Shirt Youth White/Striped New w

We need images JPEG addresses. Some have them as attribute `src`, some as attribute `data-src`. This is one way to combine the two. Make sure you understand:

In [19]:
image_files_src = [img.get('src', None) for img in images[1:]]
image_files_datasrc = [img.get('data-src', None) for img in images[1:]]
image_files = [src if datasrc is None else datasrc for src, datasrc in zip(image_files_src, image_files_datasrc)]
image_files[:5]

['https://i.ebayimg.com/thumbs/images/g/xVAAAOSwba1h-sDI/s-l300.jpg',
 'https://i.ebayimg.com/thumbs/images/g/fCMAAOSwpxJkEsIe/s-l300.jpg',
 'https://i.ebayimg.com/thumbs/images/g/xgYAAOSwRath7eHS/s-l300.jpg',
 'https://i.ebayimg.com/thumbs/images/g/1aYAAOSw~ZNjwlYj/s-l300.jpg',
 'https://i.ebayimg.com/thumbs/images/g/leIAAOSwmNVip-W6/s-l300.jpg']

Let's find a shirt's price.

Go to the url in your browser and use the code inspection tool (F12) to look interactively at the url source code. Find the element that holds price data. Notice that the price may be nested within a few levels of html tags. you are searching for the "lowest" level, the one that holds the price directly.

In our case it is a `span` element with a specific class.

What is the specific class for span elements holding the prices?

In [22]:
price_class = 's-item__price'

Get all prices:

In [20]:
price_elements = soup.find_all('span', {'class' : price_class})
print(price_elements[:5])

[<span class="s-item__price">ILS 65.34<span class="DEFAULT"> to </span>ILS 90.82</span>, <span class="s-item__price">ILS 131.01</span>, <span class="s-item__price">ILS 182.01</span>, <span class="s-item__price">ILS 81.87</span>, <span class="s-item__price">ILS 29.09<span class="DEFAULT"> to </span>ILS 36.37</span>]


From each of these `price_elements` we extract the actual price text with the `get_text` function:

In [21]:
print(price_elements[1].get_text())

ILS 131.01


Notice all prices come with the "ILS" prefix and then the number. Also you can see some prices come as a range. For this project we decided to simply take the minimum price of the range.

To do so we could split this string to its elements:

In [23]:
print(float(price_elements[1].get_text().split(' ')[1]))

131.01


Putting it in a function and getting all prices:

In [24]:
def parse_price(price_element):
    try:
        price = float(price_element.get_text().split(' ')[1])
    except:
        price = None
    return price

prices = [parse_price(price_e) for price_e in price_elements]

It's time to actually download the shirts images! The following function accepts an image file address, a shirt title and the file name for the image and attempts to download the image to the current directory with the specified file name:

In [25]:
def download_image(url, title, file_name):
    try:
        response = requests.get(url)    
    except:
        return '', ''
    with open(file_name, "wb") as file:
        file.write(response.content)
    return title, file_name

Download the first image from our page, name it 'test.jpg'. Make sure it was downloaded correctly and see what the function returns:

In [26]:
download_image(image_files[0], image_titles[0], 'test.jpg')

('Tommy Hilfiger Baby / Boys / Youth Cotton Polo Shirt NEW WITH TAG ALL Sizes !',
 'test.jpg')

We will now download all of the page's images, using a loop. 

First, create a folder named 'boys' in the current directory. You can do it right here in this notebook!

In [27]:
!mkdir boys

mkdir: cannot create directory ‘boys’: File exists


Fill in the blank to correctly create a dictionary called `images_data` which will hold the `title` of the image, its `file_name`, and the shirt's `price`. Then fill in the blank to correctly download the image with a proper file name using the `download_image` function we wrote for you.

In [31]:
from ipywidgets import IntProgress
from IPython.display import display

images_data = {'title':[None]*len(images[1:]), 'file_name':[None]*len(images[1:]),
               'price':[None]*len(images[1:])}

f = IntProgress(min = 0, max = len(images[1:])) # instantiate a progress bar
display(f) # display the bar

for i in range(len(images[1:])):
    title, file_name = download_image(image_files[i], image_titles[i], 'boys/'+str(i)+'.jpg')
    images_data['title'][i] = title
    images_data['file_name'][i] = file_name
    images_data['price'][i] = prices[i]
    f.value += 1

IntProgress(value=0, max=48)

One thing that would prove useful later on is having a dataset which summarizes all we have gathered. That's what `images_data` is for. We're going to use `pandas` to make it a `DataFrame` we can easily read and write:

In [26]:
images_data_df = pd.DataFrame(images_data)
print(images_data_df.shape)
images_data_df.head()

NameError: name 'images_data' is not defined

This was fun, we got 48 images. But we're looking to get times ~200 than that, and the same amount of shirts images for girls. 

The following function was run to get all boys shirts images. Complete it exactly as we did, but don't run it, we have the images for you!

In [34]:
run = False # change this to actually run

if run:
    boys_url = 'https://il.ebay.com/b/Boys-Tops-Shirts-T-Shirts-for-Boys/260966/bn_1642883?rt=nc&LH_ItemCondition=1000&LH_BIN=1&LH_PrefLoc=3&_pgn='
    max_pages = 400
    boys_items_data = {'title': {}, 'file_id': {}, 'price': {}}

    f = IntProgress(min = 0, max = max_pages)
    display(f)

    all_items_counter = 0

    for page_num in range(1, max_pages):
        url = boys_url + str(page_num)
        try:
            r = requests.get(url, "lxml")
        except:
            print('Stopped at page: ' + page_num)
            break
        soup = BeautifulSoup(r.content)
        ### YOUR CODE HERE: get images, image_titles, image_files, prices ###
        images = soup.find_all("img")
        image_titles = [x['alt'] for x in images[1:]]
        image_files_src = [img.get('src', None) for img in images[1:]]
        image_files_datasrc = [img.get('data-src', None) for img in images[1:]]
        image_files = [src if datasrc is None else datasrc for src, datasrc in zip(image_files_src, image_files_datasrc)]


        try:
            assert len(prices) == len(images)
        except:
            print('Found unequal number of prices in page_num % d' % page_num)
            prices = [None] * len(images)

        for i in range(len(images)):
            title, file_name = download_image(image_files[i], image_titles[i],
                                              'boys/'+'pageNum'+str(page_num)+'_pictueNum'+str(i)+'.jpg')
            boys_items_data['title'][all_items_counter + i] = title
            boys_items_data['file_id'][all_items_counter + i] = all_items_counter + i
            boys_items_data['price'][all_items_counter + i] = prices[i]
        all_items_counter += len(images)
        f.value += 1

### Wrap up
And that's it, you have used some Pandas to wrangle data and some BeautifulSoup to scrape images from the web. Good luck with the rest of the course!